# **Exploração e análise de dados de crédito com SQL**

*Esse notebook faz parte do curso SQL para análise de dados da EBAC.*

## Os dados: 

Os dados representam informações de clientes de um banco e contam com as seguintes colunas: 

* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m  = quantidade de transacoes dos ultimos 12 meses

A tabela foi criada no **AWS Athena** junto com o **S3 Bucket** com uma versão dos dados disponibilizados em: https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset


## **Exploração de dados:**

A primeira fase da análise é entender o que tem na nossa matéria prima. Vamos a exploração de dados:

**Qual a quantidade de informações temos na nossa base de dados?**

Query: SELECT count(*) FROM credito
> Reposta: 2564 linhas


**Visualização inicial dos dados:** 

Query: SELECT DISTINCT * FROM credito LIMIT 5;
![Dez primeiras linhas do dataset](https://github.com/marianeneiva/sqlEBAC/blob/main/limit10.png?raw=true)

> É possível reparar que existem alguns valores nulos na tabela que precisam ser tratados.

**Quais os tipos de cada coluna no dataset?**

**Query:** DESCRIBE credito

![Descrição dos tipos de dados](https://github.com/marianeneiva/sqlEBAC/blob/main/describe.png?raw=true)


Agora que ja entendemos quais são os tipos de dados, vamos focar em tratar os dados das variáveis não numéricas.

**Quais são os tipos de escolaridade disponíveis no dataset? Quantos registros há em cada categoria?**

**Query:** 

SELECT DISTINCT Escolaridade,
       COUNT(*) AS Quantidade
FROM Credito
GROUP BY Escolaridade
ORDER BY Quantidade DESC;

![Tipos de escolaridade](https://github.com/marianeneiva/sqlEBAC/blob/main/escolaridade.png?raw=true)

> Os dados contam com vários níveis de escolaridade e é possível perceber que temos 346 valores nulos (na) no dataset, é uma quantidade significativa e precisa ser tratada.

Será que temos mais valores nulos?

**Quais são os tipos de estado_civil disponíveis no dataset? Quantos registros há em cada categoria?**

**Query:** 

SELECT DISTINCT Estado_Civil,
       COUNT(*) AS Quantidade
FROM Credito
GROUP BY Estado_Civil
ORDER BY Quantidade DESC;

![Tipos de estado civil](https://github.com/marianeneiva/sqlEBAC/blob/main/escolaridade.png?raw=true)

> Os dados contam com os distintos registros de estado civil e é possível perceber que temos 116 valores nulos (na) no dataset, é uma quantidade significativa e precisa ser tratada.

**Quais são os tipos de salario_anual disponíveis no dataset? Quantos registros há em cada categoria?**

**Query:** 

SELECT DISTINCT Salario_Anual,
       COUNT(*) AS Quantidade
FROM Credito
GROUP BY Salario_Anual
ORDER BY Quantidade DESC;

![Tipos de salario anual](https://github.com/marianeneiva/sqlEBAC/blob/main/salario_anual.png?raw=true)

> Os dados contam com os distintos registros das faixas de salário e é possível perceber que temos 235 valores nulos (na) no dataset, é uma quantidade significativa e precisa ser tratada.


**Quais são os tipos de cartão disponíveis no dataset? Quantos registros há em cada categoria?**

**Query:** 

SELECT DISTINCT Tipo_Cartao,
       COUNT(*) AS Quantidade
FROM Credito
GROUP BY Tipo_Cartao
ORDER BY Quantidade DESC;

![Tipos de cartão](https://github.com/marianeneiva/sqlEBAC/blob/main/cartao.png?raw=true)

> Não há necessidade de tratar valores nulos.







## **Análise de dados**

Uma vez que exploramos o dados e buscamos entender quais são as informações que temos no nosso banco de dados, podemos analisar as informações para buscar entender o que está acontecendo no banco de dados. Vamos fazer perguntas:


**Nesse banco de dados, quantos clientes são homens e quantos são mulheres?**

**Query:**

SELECT DISTINCT sexo,
	count(*) as quantidade
from credito
group by sexo
order by quantidade desc


![Quantidade para cada sexo](https://github.com/marianeneiva/sqlEBAC/blob/main/faixa_sexo.png?raw=true)


> A maioria dos clientes desse banco é homem, mas a proporção não é tão descrepante.

**Queremos focar o nosso marketing de maneira adequada para nossos clientes, qual será a perfil deles?
Vamos gerar uma tabela com as informações de sexo, faixa, salarial e média de idade.**

**Query:** 

SELECT DISTINCT Sexo,
                Salario_Anual,
                AVG(Idade) AS Media_Idade,
                COUNT(*) AS Quantidade
FROM Credito
WHERE Salario_Anual <> 'na'
GROUP BY Sexo, Salario_Anual
ORDER BY Quantidade DESC;

![Média de idades por sexo](https://github.com/marianeneiva/sqlEBAC/blob/main/idade_sexopng.png?raw=true)

> Por meio dessa análise é possível perceber que a idade média se manter próxima em todas as faixas salariais. Álem disso, podemos ver que existe uma grande quantidade de mulheres ganhando abaixo de 40 mil por ano.Em contra partida, existem poucos homens ganhando menos que 40 mil por ano. Somado a isso, não existem mulheres em nosso banco de dados na faixa salarial de mais que 60 mil por ano.


**A faixa salarial impactam na quantidade de transações e limite de crédito?**

**Query:** 

SELECT DISTINCT Sexo,
                Salario_Anual,
                AVG(Valor_Transacoes_12m) AS Media_Qtd_Transacoes,
                AVG(Limite_Credito) AS Media_Limite_Credito,
                COUNT(*) AS Quantidade
FROM Credito
WHERE Salario_Anual <> 'na'
GROUP BY Sexo, Salario_Anual
ORDER BY Media_Qtd_Transacoes, Media_Limite_Credito;


![Valor transacoes](https://github.com/marianeneiva/sqlEBAC/blob/main/transacoes.png?raw=true)

> É possível ver uma forte correlação entre a faixa salarial e a quantidade de limite. Além disso, quem possue menos faixa salarial tem tendência de fazer mais transações. Os homens que recebem menos que 40 mil ao ano são os que fazem mais transações.

**Quais as características dos clientes que possuem os maiores creditos?**

**Query:** 
SELECT MAX(Limite_Credito) AS Limite_Credito,
       Escolaridade,
       Tipo_Cartao,
       Sexo
FROM Credito
WHERE Escolaridade != 'na' AND Tipo_Cartao != 'na'
GROUP BY Escolaridade, Tipo_Cartao, Sexo
ORDER BY Limite_Credito DESC
LIMIT 10;

![Valor limite](https://github.com/marianeneiva/sqlEBAC/blob/main/limite_desc.png?raw=true)

> Não parece haver um impacto da escolaridade no limite. O limite mais alto é oferecido para um homem sem educação formal. O cartão também parece não estar relacionado com a escolaridade nem com o limite. Dentre os maiores limites, encontramos clientes com cartão: gold, silver, platinum e blue



Por fim, 

**Quais as características dos clientes que possuem os menores creditos?**

**Query:** 
SELECT MAX(Limite_Credito) AS Limite_Credito,
       Escolaridade,
       Tipo_Cartao,
       Sexo
FROM Credito
WHERE Escolaridade != 'na' AND Tipo_Cartao != 'na'
GROUP BY Escolaridade, Tipo_Cartao, Sexo
ORDER BY Limite_Credito ASC;


![Valor limite](https://github.com/marianeneiva/sqlEBAC/blob/main/limite_asc.png?raw=true)

> Dessa vez conseguimos perceber que não há clientes com cartão platinum dentre os menores limites. Também foi possível perceber que a maioria dos menores limites são mulheres enquanto nos maiores limites predomina homens. 










# Conclusão

Essas foram **algumas** análises extraídas do dataset de crédito.  

Alguns insights interessantes:

- a maior parte dos clientes possui renda até 40K 
- a maior parte dos clientes é masculino
- menores faixas salariais tendem a ter maiores quantidades de transações
- a escolaridade não parece influenciar no limite nem no tipo do cartão
- os clientes com maiores limites são em sua maioria homens 
- os clientes com menores limites são em sua maioria mulheres
- dentre os menores limites não há presença de cartão platinum
- a faixa salarial impacta diretamente no limite de crédito
- nao existem clientes com salário anual acima de 60K do sexo feminino

**Uma exploração mais mais variáveis e mais dados pode explicar porque as mulheres em geral possuem menos crédito e menor faixa salarial. Isso também pode ser um problema cultural que pode ser repensado!**

